# Nueral Network
This is another model we are testing for our application. A nueral network may perform better than an XG Boost model, so we will load the same data and see if we can get better results.

In [1]:
# required imports
import pandas as pd
import pandas_profiling
import numpy as np
import gc
from sklearn.model_selection import train_test_split

# tf imports
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.optimizers import RectifiedAdam, Lookahead
from sklearn.base import BaseEstimator
from tensorflow_addons.activations import gelu


ModuleNotFoundError: No module named 'pandas_profiling'